# Neutron Star Class SPH Code

Code based on the tutorials here: 

https://philip-mocz.medium.com/create-your-own-smoothed-particle-hydrodynamics-simulation-with-python-76e1cec505f1

https://github.com/zaman13/Modeling-of-Neutron-Stars/

### Notes: 
- Calculates density from mass times the gradient of the smoothing function.
- Defines 3D Gausssian Smoothing kernel 
- Euler's Method for iterating pressure, density as a function of distance


In [161]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py

In [162]:
def EulerSolver(r,m,p,h,eos, flag):
    """ Euler Solver which iterates P """
    dp_dr = eos.dp_dr # extract the relevant dp_dr from EOS.
    return p + dp_dr(r,m,p,flag)*h

def getPressure(rho, eos, star, tol = 9e-5):
    """ Takes in a star and an eos. Calcualates the pressure at each point in the star. Returns a vector with shape (num_points x 1)."""
    ## Exctract relavent values from the star & EOS
    N = star.num_points
    m = star.m
    k = eos.k
    n = eos.n
    rho_s = eos.rho_s
    #############################################################
    ni = initial_n(eos)
    
    r = np.linspace(0,15,N) 
    r[0] = 0
    p = np.zeros(N) # each data point has a pressure.
    p[0] = 363.44 * (ni**2.54)/rho_s
    rh = np.zeros(N)
    rh[0] = 1
    
    h = r[1]-r[0] # we have re-defined the smoothing length here.
    
    for i in range(0,N-1): # for each point
        p[i+1] = EulerSolver(r[i],m,p[i],h,eos,0) # Euler's Method to update pressure
        rh[i+1] = rho_NS(r[i])
        if p[i+1] < tol:
            rf = r[i]
            break       
    print("P <", tol, "found after", i, "runs")
    
    p = p[0:i+2] # Update the pressure of each point
    rh = rh[0:i+2]
    r = r[0:i+2]
    
    ## p is not going to have the same shape as the star points!!!! 
    ### Need to extrapolate the pressures to the points where the stars are, then return p!!! 
    return p

In [155]:
def getAcc(star, m,h, eos, lmbda, nu):
    """ Calculate the acceleration on each SPH particle given current positions and velocities.
    
    'pos' contains particle coordinates (N x 3 matrix)
    'vel' contains particle velocities (N x 3 matrix)
    returns a containing accelerations (another N x 3 matrix)
    
    m = particle mass, h = smoothing length, k = equation of state constant, n = polytropic index, lmbda = external force constant, nu = viscosity. 
    
    """ 
    # Exctract positions and velocities of points from the star
    pos = star.pos 
    vel = star.vel
    
    N = pos.shape[0] # Number of particles
    rho = getDensity(pos, pos, m, h) # Reconstruct the densities at the position of each particle
    P = getPressure(rho, eos, star) # Get the pressures corresponding to those densities
    dx, dy, dz = getPairwiseSeparations( pos, pos ) # Get pairwise distances
    dWx, dWy, dWz = gradW( dx, dy, dz, h )# Get pairwise gradients
    
    # Current error 
    
    
    ax = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWx, 1).reshape((N,1)) # Add x Pressure contribution to accelerations
    ay = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWy, 1).reshape((N,1)) # Add y Pressure contribution to accelerations
    az = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWz, 1).reshape((N,1)) # Add z Pressure contribution to accelerations
    a = np.hstack((ax,ay,az)) # pack together the acceleration components
    
    ##### Extra forces
    a -= lmbda * pos # Add external potential force
    a -= nu * vel # Add viscosity
    
    #####
    return a

### Equations of State

In [156]:
rho_s = 1665.3 #  Central density (density at r = 0)
def rho_NS(p): 
    """ density from pressure NS, source: https://github.com/zaman13/Modeling-of-Neutron-Stars/ """
    n = (p*rho_s/363.44)**(1./2.54)
    return (236. * n**2.54 + n *mn)/rho_s 

def dp_dr_NS(r,m,p, Classical = False): 
    """ pressure dependence on radius NS, source: https://github.com/zaman13/Modeling-of-Neutron-Stars/ """
    if Classical:                              # classical model
        y = -m*rho_NS(p)/(r**2 + 1e-20)
    else:                                      # relativistic model
        rh = rho_NS(p)                            
        y = -(p+rh)*(p*r**3 + m)/(r**2 - 2*m*r + 1e-20)
    return y

def dm_dr_NS(r,m,p):
    return rho_NS(p)*r**2

hc = 197.327           # Conversion factor in MeV fm (hut * c)
G = hc * 6.67259e-45   # Gravitational constant
Ms = 1.1157467e60      # Mass of the sun in kg
mn = 938.926           # Mass of neutron in MeV c^-2

def initial_n(eos): # # Function for determining initial value of n(r=0). I don't know what n is honestly...
    n = 1
    err = 1
    tol = 1e-15
    count = 0
    
    rho_s = eos.rho_s
    # Newton-Raphson method
    while err > tol : 
        count += 1
        fn = n*mn + 236*n**(2.54) - rho_s
        dfn = mn + 236*2.54*n**(1.54)
        temp = n - fn/dfn
        err = np.abs(n-temp)
        n = temp
    print("Newton-Raphson Converged after ", count, "iterations")
    return n

class EOS: 
    def __init__(self, name, P= None, dp_dr= None, notes = None):
        """ Initialize an EOS Object to use multiple times.
        If the name is regocnized, the EOS will already have all of it's properties built in.  If not, you can specify the properties. 
        """
        self.name = name
        if self.name == 'std':
            rho_s = 1665.3 #  Central density (density at r = 0)
            self.rho_s = rho_s
            self.M0 = (4*3.14159265*(G**3)*rho_s)**(-0.5)
            self.R0 =  G*self.M0
            self.rho = lambda x : rho_NS(x)
            self.dp_dr = lambda r,m,p, flag: dp_dr_NS(r,m,p, flag)
            self.dm_dr = lambda r, m, p: dm_dr_NS(r,m,p)
            self.notes = 'Standard EOS.'
            self.h = 0.1 # smoothing length
            self.k = 0.1 # equation of state constant
            self.n = 1 # polytropic index
            n = self.n
            self.lmbda = lambda M,R : 2*self.k*(1+n)*np.pi**(-3/(2*n)) * (M*gamma(5/2+n)/R**3/gamma(1+n))**(1/n) / R**2  #is external force constant, ~ 2.01
        
        else:
            self.P = P
            self.dp_dr = dp_dr
            self.notes = notes

    def P_eos(self, rho): 
        func = self.P
        return func(rho)

## Star Class 
Notes: 

#### The Gaussian Smoothing function is used to smooth out the point values, extrapolating them to other positions.
- Given the distribution of particles, we can reconstruct the density at any location using the smoothing kernels. For example, the density at each SPH particle location is a sum over all the particles with weighting set by the distances between particles and the smoothing kernel:

In [157]:
def W(x,y,z,h):
    """ Defined Gausssian Smoothing kernel (3D).
    (x is a vector/matrix of x positions, y is a vector/matrix of y positions. 
    z = vector/matrix of z positions, h = smoothing length, w = evaluated smoothing function."""
    r = np.sqrt(x**2 + y**2 + z**2) 
    return (1.0 / (h*np.sqrt(np.pi)))**3 * np.exp( -r**2 / h**2) # return w
    
def gradW(x,y,z,h):
    """ Gradient of smoothing kernel. We can reconstruct the density at any location using the smoothing kernels.
    (x is a vector/matrix of x positions, y is a vector/matrix of y positions, z is a vector/matrix of z positions, h is the smoothing length, wx, wy, wz  is the evaluated gradient)"""
    r = np.sqrt(x**2 + y**2 + z**2)  
    n = -2 * np.exp( -r**2 / h**2) / h**5 / (np.pi)**(3/2)
    return n*x, n*y, n*z  # (gradient in the x, y, and z directions)
    
def getPairwiseSeparations(ri, rj):
    """ Just finds Cartesian Pairwise Separations between 2 points. 
    ri is an M x 3 matrix of positions, 
    rj    is an N x 3 matrix of positions 
    dx, dy, dz   are M x N matrices of separations. """
    M = ri.shape[0]
    N = rj.shape[0]
    rix = ri[:,0].reshape((M,1)) # positions ri = (x,y,z)
    riy = ri[:,1].reshape((M,1))
    riz = ri[:,2].reshape((M,1))
    rjx = rj[:,0].reshape((N,1)) # other set of points positions rj = (x,y,z)
    rjy = rj[:,1].reshape((N,1))
    rjz = rj[:,2].reshape((N,1))
    return rix - rjx.T , riy - rjy.T, riz - rjz.T # (dx, dy, dz) # return matrices that store all pairwise particle separations: r_i - r_j

def getDensity(r, pos, m, h ):
    """ Reconstruct the density at any location based on the sph points.
            r     is an M x 3 matrix of sampling locations
            pos   is an N x 3 matrix of SPH particle positions
          
          dx, dy, dz are all M x N matrices. (Seperations). 
          - Sum over all particle interactions with the points to get the density at the locations in r.
          
          m = particle mass, h = smoothing length """
    
    M = r.shape[0]
    dx, dy, dz = getPairwiseSeparations( r, pos );
    rho = np.sum( m * W(dx, dy, dz, h), 1 ).reshape((M,1))
    return rho


In [158]:
class NS:
    def __init__(self, tag, mass = 1, radius = 1, num_points = 300):
        """
        NEUTRON STAR OBJECT
        
        A neutron star has the following attributes: 
            - tag, mass, radius, num_points  (eos?)
        
        The each point has the following attributes: 
            - position (N x 3), vel (N x 3), acc (N x 3), density (N x 1), pressure (N x 1).        
        
        Goal: we want to give this star it's overall attributes and calculate the point attributes automatically.
        
        """
        self.tag = tag # "Name" of the star, just a string for identification
        self.radius = radius # star radius in km
        self.mass = mass # star mass in solar Masses
        self.radius_unit = 1 # if the radius of the star is given in units other than kilometers, you can give it a scale factor here.    
        
        self.num_points = num_points 
        self.m = mass/num_points
        
        # for each point in the star, initialize random positions and velocities.
        np.random.seed(42) # set the random number generator seed
        self.pos = np.random.randn(self.num_points,3)   # randomly select positions and velocities from initialized seed
        self.vel = np.zeros(self.pos.shape)    # randomly select positions and velocities 
        # positions and velocities are N x 3 matrices.
        self.acc = np.zeros(self.pos.shape)  # initially we haven't updated the accelerations of those random points for the star.
        self.rho = None
        self.density = None
        
        
    def __str__(self):
        return "Name: '" + self.tag + "', Mass: " + str(self.radius) + " SMs, "+ "Radius: " + str(self.mass*self.radius_unit) + " km"
    
    def get_Acc(self, eos):
        a = getAcc(self, self.mass, h, eos, lmbda, nu) # calculate initial gravitational accelerations
        self.acc = a
        return self.acc
        # acc is also an N x 3 matrix (so new we have pos, vel, acc for x,y,z.)
    
    def initialize_mp_distributions(self): 
        """ Initializes an array of M and p based off of the mass and radius of the star, and the EOS. The star only needs to do this once. """
    
    def reset(self, num_points): 
        """ Resets the star with a new number of points. Randomized positions of the new points. """
        self.num_points = num_points
        np.random.seed(42) # set the random number generator seed
        self.pos = np.random.randn(self.num_points,3)   # randomly select positions and velocities from initialized seed
        self.vel = np.zeros(self.pos.shape)    # randomly select positions and velocities 
        # positions and velocities are N x 3 matrices.
        self.acc = None # initially we haven't updated the accelerations of those random points for the star.
        self.rho = None
        self.density = None    
        
    def simulate(self, eos, dt = 0.04, tEnd = 12):
        N = self.num_points # Number of particles
        M = self.mass # total star mass (Solar Masses)
        R = self.radius # star radius (km)
        k = eos.k # equation of state constant
        n = eos.n # polytropic index
        h = 0.05 # from previous results, this seems like a good smoothing length.
        nu = 1 # damping
        lmbda = eos.lmbda(M,R)
        t = 0      # current time of the simulation
        Nt = int(np.ceil(tEnd/dt)) # number of timesteps

        for i in range(Nt): # Star Simulation Time Iteration Main Loop
            self.vel += self.acc * dt/2 # (1/2) kick # adds acceleration
            self.pos += self.vel * dt # particle motion
            self.acc = getAcc(self, self.m, h, eos, lmbda, nu) # get new Nx3 matrix of accelerations based on new pos,v
            self.vel += self.acc * dt/2 # (1/2) kick # not sure why we add accelleration again.
            t += dt # update time
            if t % 100:
                print("time: " , t)
            self.rho = getDensity(self.pos, self.pos, self.m, h) # get updated density for each point based on m, h. 
        
        # At this point, the star has been simulated. Now you can plot the star's points, density, etc...

In [159]:
### Simulation

# Create a star

star_test = NS("star_test") 
print(star_test)

# Create an EOS 

my_EOS = EOS('std')
print(my_EOS.notes)


Name: 'star_test', Mass: 1 SMs, Radius: 1 km
Standard EOS.


In [160]:
star_test.simulate(my_EOS)

Newton-Raphson Converged after  5 iterations
P < 9e-05 found after 0 runs


ValueError: operands could not be broadcast together with shapes (2,) (1,300) 